In [21]:
import sys
from pathlib import Path

# add parent folder to the path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

___

In [22]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader # load batches to the network

# architectures
from src.models import (
    SimpleNet,
    Rodan
)

# loss function and dataset loader
from src.dataloader import DatasetONT # custom loader (used with DataLoader)
from src.loss_functions import ctc_label_smoothing # custom CTC loss function

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

___
### Check dataset

In [8]:
output_simplenet = 501
dataset_example = DatasetONT(recfile="../data/subsample_val.hdf5", output_network_len=output_simplenet)
loader = iter(DataLoader(dataset_example, batch_size=2, shuffle=True))
x,y, input_len, target_len = next(loader)
print("Input [batch size, channels, length]:" , x.shape)
print("Output [batch size, length]:" , y.shape)
print("CTC Loss")
print("Input CTC Loss [batch size]", input_len.shape)
print("Target CTC Loss [batch size]", target_len.shape)

Input [batch size, channels, length]: torch.Size([2, 1, 4096])
Output [batch size, length]: torch.Size([2, 271])
CTC Loss
Input CTC Loss [batch size] torch.Size([2])
Target CTC Loss [batch size] torch.Size([2])


In [9]:
input_len, target_len

(tensor([501, 501]), tensor([271, 271]))

___
SimpleNet

In [10]:
simplenet = SimpleNet(n_channels = 1, n_classes = 271)
simplenet.eval()

SimpleNet(
  (conv1): Conv1d(1, 20, kernel_size=(20,), stride=(2,))
  (relu1): ReLU()
  (maxpool1): MaxPool1d(kernel_size=10, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(20, 50, kernel_size=(5,), stride=(1,))
  (relu2): ReLU()
  (maxpool2): MaxPool1d(kernel_size=10, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50, out_features=5, bias=True)
  (relu3): ReLU()
)

In [11]:
output = simplenet(x)
output.shape

torch.Size([501, 2, 5])

___
RODAN

In [12]:
# from collections import namedtuple
# DEFAULTCONFIG = dict(
#     vocab=["<PAD>", "A", "C", "G", "T"],
#     activation_layer="mish", # options: mish, swish, relu, gelu
#     sqex_activation="mish", # options: mish, swish, relu, gelu
#     dropout=0.1,
#     sqex_reduction=32
# )

# Config=namedtuple("CONFIG",["vocab", "activation", "sqex_activation", "dropout", "sqex_reduction"])

# rodan = Rodan(config=Config(*DEFAULTCONFIG.values()))
# rodan.eval()

In [13]:
# x,y, input_len, target_len = next(loader)
# print("Input [batch size, channels, length]:" , x.shape)
# print("Input [batch size, length]:" , y.shape)

# output_rodan = rodan(x)
# print("output rodan:", output_rodan.shape)

___
## Training

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    """
    training function
    """
    size = len(dataloader) # number of datapoints in the dataset
    model.train() # set model in training mode

    for batch, (X,y, input_len, target_len) in enumerate(dataloader):
        X, y, input_len, target_len = X.to(device), y.to(device), input_len.to(device), target_len.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y, input_lengths=input_len, target_lengths=target_len)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0: 
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [15]:
def test(dataloader, model, loss_fn, model_metadata=None):
    """
    test function

    Accuracy is measured here
    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # let the model know that is in evaluation model
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X,y, input_len, target_len) in enumerate(dataloader):
            X, y, input_len, target_len = X.to(device), y.to(device), input_len.to(device), target_len.to(device)

            # Compute prediction error
            pred = model(X)
            test_loss += loss_fn(pred, y, input_lengths=input_len, target_lengths=target_len).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
# architecture
model = simplenet
# simplenet params
simplenet_output_len = 501
rodan_output_len = 420 

# params
epochs = 5

# dataset
dataset_train = DatasetONT(recfile="../data/subsample_train.hdf5", output_network_len=simplenet_output_len)
dataset_val   = DatasetONT(recfile="../data/subsample_val.hdf5", output_network_len=simplenet_output_len)

dataloader_train = DataLoader(dataset_val, batch_size=16, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=16, shuffle=False)

# loss function and optimizer
loss_fn = nn.CTCLoss() #ctc_label_smoothing # 
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [18]:
loss_fn = nn.CTCLoss() #ctc_label_smoothing # 

for batch, (X,y, input_len, target_len) in enumerate(dataloader_train):
    X, y, input_len, target_len = X.to(device), y.to(device), input_len.to(device), target_len.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y, input_lengths=input_len, target_lengths=target_len)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [17]:
pred.shape, y.shape, input_len

(torch.Size([501, 16, 5]),
 torch.Size([16, 271]),
 tensor([561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561, 561,
         561, 561]))

In [26]:
torch.from_numpy(np.array(561))

tensor(561)

In [19]:
# run training 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader_train, model=model, loss_fn=loss_fn, optimizer=optimizer)
    test(dataloader_val, model=model, loss_fn=loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:     nan [   16/   13]


RuntimeError: The size of tensor a (5) must match the size of tensor b (271) at non-singleton dimension 1

In [23]:
# rodan(x)
x.shape

torch.Size([2, 1, 4096])

___
## basecalling 


In [ ]:
import numpy as np
from fast_ctc_decode import beam_search, viterbi_search
alphabet = "NACGT"

In [ ]:
posteriors = output_rodan[:,1,:].detach().numpy()
seq, path = viterbi_search(posteriors, alphabet)
seq

'T'

In [ ]:

posteriors = np.random.rand(100, len(alphabet)).astype(np.float32)
posteriors

In [ ]:
seq, path = viterbi_search(posteriors, alphabet)

In [ ]:
seq, len(seq)


('GATCTCTATATGTGTATCACAGCAGTCATCTCATCGACGCACTCACT', 47)

In [ ]:
path

[0,
 2,
 3,
 6,
 9,
 14,
 16,
 17,
 18,
 20,
 21,
 22,
 23,
 24,
 25,
 28,
 29,
 32,
 35,
 37,
 40,
 41,
 43,
 46,
 48,
 50,
 52,
 53,
 54,
 56,
 58,
 59,
 60,
 61,
 64,
 65,
 66,
 68,
 70,
 72,
 75,
 78,
 82,
 83,
 84,
 85,
 86,
 88,
 90,
 91,
 93,
 94,
 96,
 97,
 98,
 99]

In [ ]:
seq, path = beam_search(posteriors, alphabet, beam_size=5, beam_cut_threshold=0.1)
seq, len(seq)

('GATCTCTATATGTGTATCACAGCAGTCATCTCATCGACGCACTCACT', 47)